<a href="https://colab.research.google.com/github/Gheitanchi/PromptEng/blob/main/HF_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (Optional) Lesson 14: Deploy ML Models on 🤗 Hub using Gradio

- Welcome to the last lesson - ML deployment using 🤗 Hub and Gradio libraries.
- This lesson is optional.  You can watch the video first to see a walkthrough of how to deploy to Hugging Face Spaces.
- If you would like to follow along or deploy to Hugging Face Spaces later, you can do so by creating a free account on https://huggingface.co/
- You are not required to create an account to complete this lesson, as this lesson contains screenshots and instructions for how to deploy, but does not have any code that requires you to have a Hugging Face account.

- In the classroom, the libraries are already installed for you.
- If you would like to run this code on your own machine, you can install the following:

```
    !pip install transformers
    !pip install gradio
    !pip install gradio_client
```

- Note that if you run into issues when making an API call to your own space, you can try to upgrade your version of gradio_client:

```
pip install -U gradio_client
```

- Here is some code that suppresses warning messages.

In [ ]:
from transformers.utils import logging
logging.set_verbosity_error()

import warnings
warnings.filterwarnings("ignore",
                        message="Using the model-agnostic default `max_length`")

## 🤗 Spaces

- You can create an account on hugging face from [here](https://huggingface.co), to follow the instructions provided in the video.

### App Demo: Image Captioning

- Load the model and create an app interface using Gradio to perform Image Captioning.

### Troubleshooting Tip
- Note, in the classroom, you may see the code for creating the Gradio app run indefinitely.
  - This is specific to this classroom environment when it's serving many learners at once, and you won't wouldn't experience this issue if you run this code on your own machine.
- To fix this, please restart the kernel (Menu Kernel->Restart Kernel) and re-run the code in the lab from the beginning of the lesson.

In [ ]:
import os
import gradio as gr
from transformers import pipeline

In [ ]:
pipe = pipeline("image-to-text",
                model="./models/Salesforce/blip-image-captioning-base")

In [ ]:
def launch(input):
    out = pipe(input)
    return out[0]['generated_text']

In [ ]:
iface = gr.Interface(launch,
                     inputs=gr.Image(type='pil'),
                     outputs="text")

In [ ]:
iface.launch(share=True,
             server_port=int(os.environ['PORT1']))

- Close the demo port.

In [ ]:
iface.close()

### Close the app
- Please remember to call `.close()` on the Gradio app before continuing.

### Deploying to Hugging Face Spaces

- Go to [https://huggingface.co/spaces](https://huggingface.co/spaces)![](./images_app/create_new_space_00.png)


- Click the button "create new space".

![](./images_app/create_new_space_01.png)

- Give the space a name, such as "blip-image-captioning".
- Choose a license, such as Apache 2.0
- For "Select the Space SDK", click "Gradio".
- For Hardware, choose the default free option: "CPU Basic"


![](./images_app/create_new_space_02.png)
- Leave it as "public"
- Click "create space".

![](./images_app/get_started_01.png)

- You will see a new page with instructions for how to get started by cloning and updating a GitHub repo.
- You can also add the required files directly in the web browser if you'd like to get a small app running quickly.  Click on "Files" at the top.

![](./images_app/files_01.png)

- Click on "+ Add file"->"Create new File".

### Add requirements.txt


![](./images_app/requirements_01.png)

- Add a file called requirements.txt.
- Paste in the following:

```
transformers
torch
gradio
```

- Leave "Commit Directly to the main branch" selected.
- Click "commit new file to main".

### Add app.py

![](./images_app/app_01.png)

- In the textbox "Name Your File", type "app.py"
- In the textbox for your code, paste in the code that you ran above, or copy this block below:



```Python
import gradio as gr
from transformers import pipeline

pipe = pipeline("image-to-text",
                model="Salesforce/blip-image-captioning-base")


def launch(input):
    out = pipe(input)
    return out[0]['generated_text']

iface = gr.Interface(launch,
                     inputs=gr.Image(type='pil'),
                     outputs="text")

iface.launch()
```
- Notice that `iface.launch()` does not have `share=True`

![](./images_app/app_02.png)

- Leave "Commit Directly to the main branch" selected.
- Click "Commit new file to main".

### View the app
![](./images_app/app_tab.png)

- You will see that the app is still "Building" for a few minutes.
- You can click on the "App" menu to the left of the "Files" menu to see the console as the space is being built.
![](app_tab.png)

![](./images_app/app_tab_03.png)

- When the build is done, you'll see your app!
- At the bottom, you can click "Use via API" to see sample code that you can use to use your model with an API call.

![](./images_app/api_01.png)

- You can run the pip install if you haven't already done so.
- In the classroom, gradio_client should already be installed for you.
- Copy the sample code, which will look something like this:

```Python
from gradio_client import Client

client = Client("eddyS/blip-image-captioning-2")
result = client.predict(
		"https://raw.githubusercontent.com/gradio-app/gradio/main/test/test_files/bus.png",	# filepath  in 'input' Image component
		api_name="/predict"
)
print(result)
```

- Note, you can replace the string within `client.predict()` with a string that points to a local file.
- In the classroom, there are two image files that you can use.
  - "kittens.jpg"
  - "huggingface_friends.jpg"
  - Feel free to upload your own to the file directory.

So your code may look like this:
```Python
from gradio_client import Client

client = Client("eddyS/blip-image-captioning-2")
result = client.predict(
		"kittens.jpg",
		api_name="/predict"
)
print(result)
```

- Inspect the information in the API.

```Python
client.view_api()
```
- The output may look like this:


```
Client.predict() Usage Info
---------------------------
Named API endpoints: 1

 - predict(input, api_name="/predict") -> output
    Parameters:
     - [Image] input: filepath
    Returns:
     - [Textbox] output: str

```

### (Optional) Access to your private space as an API.

You can make your space private, so that it can be accessed with an access token.

To make the space private click on the "Settings" menu at the top.  Scroll down until you see "Change space visibility" and click the "Make private" button.

![](./images_app/private_01.png)

### Get an access token
- To get an access token, go to your profile (click on your profile icon).
- On your profile page, click the "Settings" button on the left.

![](./images_app/profile_01.png)

- In your profile settings, on the left side menu, click "Access Tokens".
- Click "New token".

![](./images_app/access_tokens_01.png)

- In the pop-up, give a description of what the token is for.
- You can leave it as "read" (the other option is "write").
- Click "create new token".  

![](./images_app/new_token_01.png)

- You can copy the access token.

![](./images_app/access_tokens_02.png)

You can modify the API call to include your access token.

```Python
from gradio_client import Client

client = Client("eddyS/blip-image-captioning-2",
                hf_token=hf_access_token
               )
result = client.predict(
		"kittens.jpg",
		api_name="/predict"
)
print(result)
# client = Client("abidlabs/whisper-large-v2",
)
```

### Saving your access token securely
- It's recommended that you not hard code the access token.

```Python
HF_TOKEN="abc1234" # not recommended
```

- You can save your access token to a file ".env"

```
HF_ACCESS_TOKEN="abc123"
```

Then access that environment variable with the `dotenv` library

```Python
# !pip install python-dotenv # install library
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())
hf_access_token = os.getenv("HF_ACCESS_TOKEN")
```

### GPU Zero Space
- [ZeroGPU Explorers](https://huggingface.co/zero-gpu-explorers): A place to spin free GPUs on demand for your spaces.

![](./images_app/gpu_01.png)

- You can click "request to join this org".
- It may take a few days/weeks for this to be approved.